In [85]:
import pandas as pd
from duckduckgo_search import DDGS
from langdetect import detect
from googletrans import Translator

def search_duckduckgo(query, max_results=5):
    results = DDGS().text(
        keywords=query,
        region='wt-wt',
        safesearch='off',
        timelimit='none',
        max_results=max_results
    )
    return pd.DataFrame(list(results))

def is_twitter_link(url):
    return 'twitter.com' in url.lower()

def is_game_related(text):
    # Mots-clés liés aux jeux vidéo en plusieurs langues
    game_keywords = {
        'en': ['game', 'video game', 'developer', 'studio', 'gaming', 'indie'],
        'fr': ['jeu', 'jeu vidéo', 'développeur', 'studio', 'gaming'],
        'es': ['juego', 'videojuego', 'desarrollador', 'estudio', 'gaming'],
        'de': ['spiel', 'videospiel', 'entwickler', 'studio', 'gaming'],
        'it': ['gioco', 'videogioco', 'sviluppatore', 'studio', 'gaming'],
        'ja': ['ゲーム', 'ビデオゲーム', '開発者', 'スタジオ', 'ゲーミング'],
    }
    
    translator = Translator()
    
    try:
        # Détecter la langue du texte
        lang = detect(text)
        
        # Si la langue n'est pas dans notre liste, traduire en anglais
        if lang not in game_keywords:
            text = translator.translate(text, dest='en').text
            lang = 'en'
        
        # Vérifier si l'un des mots-clés est présent dans le texte
        return any(keyword.lower() in text.lower() for keyword in game_keywords[lang])
    except:
        # En cas d'erreur (par exemple, texte trop court), on retourne False
        return False

def analyze_results(df):
    for index, row in df.iterrows():
        if is_twitter_link(row['href']):
            print(f"Lien Twitter trouvé : {row['href']}")
            if is_game_related(row['body']):
                print("Ce compte semble être lié aux jeux vidéo.")
                return row['href']
            else:
                print("Ce compte ne semble pas être lié aux jeux vidéo.")
    
    print("Aucun lien Twitter pertinent n'a été trouvé.")
    return None

# Exemple d'utilisation
search_query = "AlleyInn twitter"
results_df = search_duckduckgo(search_query)

print(f"Nombre de résultats trouvés : {len(results_df)}")
relevant_twitter_link = analyze_results(results_df)

if relevant_twitter_link:
    print(f"Lien Twitter pertinent trouvé : {relevant_twitter_link}")
else:
    print("Aucun lien Twitter pertinent n'a été trouvé.")





    

Nombre de résultats trouvés : 5
Lien Twitter trouvé : https://mobile.twitter.com/alley_inn
Ce compte semble être lié aux jeux vidéo.
Lien Twitter pertinent trouvé : https://mobile.twitter.com/alley_inn


/Users/juliebelzanne/Library/Caches/pypoetry/virtualenvs/newgame-newtweet-icRyx73a-py3.12/lib/python3.12/site-packages/langdetect/detector_factory.py:91: RuntimeWarning: coroutine 'main' was never awaited
  prob = 1.0 * profile.freq.get(word) / profile.n_words[length - 1]


In [86]:
import pandas as pd
from duckduckgo_search import DDGS
from langdetect import detect
from googletrans import Translator
import re

def search_duckduckgo(query, max_results=5):
    results = DDGS().text(
        keywords=query,
        region='wt-wt',
        safesearch='off',
        timelimit='7d',
        max_results=max_results
    )
    return pd.DataFrame(list(results))

def is_twitter_link(url):
    return 'twitter.com' in url.lower()

def is_game_related(text):
    game_keywords = {
        'en': ['game', 'video game', 'developer', 'studio', 'gaming'],
        'fr': ['jeu', 'jeu vidéo', 'développeur', 'studio', 'gaming'],
        'es': ['juego', 'videojuego', 'desarrollador', 'estudio', 'gaming'],
        'de': ['spiel', 'videospiel', 'entwickler', 'studio', 'gaming'],
        'it': ['gioco', 'videogioco', 'sviluppatore', 'studio', 'gaming'],
        'ja': ['ゲーム', 'ビデオゲーム', '開発者', 'スタジオ', 'ゲーミング'],
    }
    
    translator = Translator()
    
    try:
        lang = detect(text)
        if lang not in game_keywords:
            text = translator.translate(text, dest='en').text
            lang = 'en'
        return any(keyword.lower() in text.lower() for keyword in game_keywords[lang])
    except:
        return False

def extract_twitter_handle(url):
    match = re.search(r'twitter\.com/(\w+)', url)
    if match:
        return '@' + match.group(1)
    return None

def get_game_studio_twitter(studio_name):
    search_query = f"{studio_name} twitter"
    results_df = search_duckduckgo(search_query)
    
    for index, row in results_df.iterrows():
        if is_twitter_link(row['href']):
            handle = extract_twitter_handle(row['href'])
            if handle and is_game_related(row['body']):
                return handle
    
    return None

# Exemple d'utilisation
studio_name = "AlleyInn"
twitter_handle = get_game_studio_twitter(studio_name)

if twitter_handle:
    print(f"Le compte Twitter de {studio_name} est : {twitter_handle}")
else:
    print(f"Aucun compte Twitter pertinent n'a été trouvé pour {studio_name}")

Le compte Twitter de AlleyInn est : @alley_inn
